In [5]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import ttk

class ExpenseTracker:
    def __init__(self, root):
        self.root = root
        self.root.title("Expense Tracker")

        # Charger les dépenses existantes à partir du fichier CSV
        try:
            self.expenses_df = pd.read_csv("depenses.csv", parse_dates=["Date"])
        except FileNotFoundError:
            self.expenses_df = pd.DataFrame(columns=["Date", "Montant", "Description"])

        # Créer les champs de saisie et les boutons
        style = ttk.Style()
        style.configure('TLabel', font=('Arial', 9), padding=8)
        style.configure('TButton', font=('Arial', 9), padding=8)
        style.configure('TEntry', font=('Arial', 9), padding=8)

        self.date_label = ttk.Label(root, text="Date (YYYY-MM-DD)")
        self.date_label.pack()
        self.date_entry = ttk.Entry(root)
        self.date_entry.pack()

        self.amount_label = ttk.Label(root, text="Amount")
        self.amount_label.pack()
        self.amount_entry = ttk.Entry(root)
        self.amount_entry.pack()

        self.description_label = ttk.Label(root, text="Description (optional)")
        self.description_label.pack()
        self.description_entry = ttk.Entry(root)
        self.description_entry.pack()

        self.add_button = ttk.Button(root, text="Add Expense", command=self.add_expense)
        self.add_button.pack()

        self.view_button= ttk.Button(root, text="View Expenses", command=self.view_expenses)
        self.view_button.pack()

        self.visualize_button = ttk.Button(root, text="Visualize Expenses", command=self.visualize_expenses_tk)
        self.visualize_button.pack()

    def add_expense(self):
        date = self.date_entry.get()
        amount_str = self.amount_entry.get()
        description = self.description_entry.get()

        if date and amount_str:
            try:
                amount = float(amount_str)
            except ValueError:
                messagebox.showerror("Error", "Invalid amount")
                return

            new_expense = pd.DataFrame({"Date": [date], "Montant": [amount], "Description": [description]})
            self.expenses_df = pd.concat([self.expenses_df, new_expense], ignore_index=True)

            self.date_entry.delete(0, tk.END)
            self.amount_entry.delete(0, tk.END)
            self.description_entry.delete(0, tk.END)

            # Mettre à jour la visualisation après l'ajout d'une nouvelle dépense
            self.visualize_expenses_tk()

            self.save_data()
        else:
            messagebox.showerror("Error", "Date and amount are required.")

    def view_expenses(self):
        messagebox.showinfo("Expenses", self.expenses_df.to_string())

    def visualize_expenses(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.expenses_df.groupby("Date").sum()["Montant"])
        plt.xlabel("Date")
        plt.ylabel("Amount")
        plt.title("Monthly Expenses")
        plt.show()

    def visualize_expenses_tk(self):
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111)
        ax.plot(self.expenses_df.groupby("Date").sum()["Montant"])
        ax.set_xlabel("Date")
        ax.set_ylabel("Montant")
        ax.set_title("Monthly Expenses")

        # Effacer les données existantes avant d'ajouter les nouvelles données
        ax.cla()
        ax.plot(self.expenses_df.groupby("Date").sum()["Montant"])

        canvas = FigureCanvasTkAgg(fig, master=self.root)
        canvas.draw()
        canvas.get_tk_widget().pack()

    def save_data(self):
        self.expenses_df.to_csv("depenses.csv", index=False)

    def on_closing(self):
        self.save_data()
        self.root.destroy()

root = tk.Tk()
app = ExpenseTracker(root)
root.protocol("WM_DELETE_WINDOW", app.on_closing)
root.mainloop()
